In [1]:
import os

# Move to Thesis directory (two levels up)
os.chdir(os.path.abspath(os.path.join("..", "..")))

# Move to model/src if it exists
model_dir = os.path.join(os.getcwd(), "model", "src")
if os.path.exists(model_dir):
    os.chdir(model_dir)

print("Current Directory:", os.getcwd())


Current Directory: c:\Users\1176153\Downloads\github\Thesis\model\src


# Imports

In [2]:
import selenium
import requests
import pandas as pd
import numpy as np 
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import os
import re
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup
from webdriver_manager.chrome import ChromeDriverManager


## Loading dictionary with all the program links

In [3]:
dict_programs_links = pd.read_pickle(r"../../data/Webscrapping/programs_links.pkl")
dict_programs_links

{'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/': ['https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/data-science/',
  'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/information-management/',
  'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/information-systems/',
  'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/applications-to-the-special-contest-of-access-and-entry-change-of-institution-course-pair-academic-year-2024-2025/',
  'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/applications-to-the-special-contest-for-over-23-years-old-access-entry-academic-year-2024-2025/',
  'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/international-students/'],
 'https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/': ['https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-p

In [4]:
if len(dict_programs_links) > 1:
    # Convert dictionary items to a list and get the second key-value pair
    items = list(dict_programs_links.items())
    third_url, third_filtered_links = items[2]
    
    # Print the second URL and filtered links
    print(f"Second URL: {third_url}")
    print(f"Filtered links for the second URL: {third_filtered_links}")
elif dict_programs_links:
    print("The dictionary does not have a second item.")
else:
    print("The dictionary is empty.")

Second URL: https://www.novaims.unl.pt/en/education/programs/doctoral-program-in-information-management/doctoral-program-in-information-management/
Filtered links for the second URL: ['https://www.novaims.unl.pt/en/education/programs/doctoral-program-in-information-management/doctoral-program-in-information-management/faculty/']


## We have only 1 link for PhD program so we don't need a complicated pipeline

### PhD

I'm ignoring at the moment schedules and timetables

In [8]:
# Directory for storing extracted text
PhD_dir = r"..\..\data\Webscrapping\PhD_degrees"
os.makedirs(PhD_dir, exist_ok=True)  # Ensure directory exists

def sanitize_filename(filename):
    """Sanitize the filename to remove any invalid characters."""
    # Replace invalid characters with underscores
    return re.sub(r'[<>:"/\\|?*]', '_', filename)

In [9]:
# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

PhD_url = third_url

# Open webpage
driver.get(PhD_url)
time.sleep(5)  # Wait for the page to load

# Extract the <main> element
main_element = driver.find_element(By.TAG_NAME, "main")

# Get the inner HTML of <main>
main_html = main_element.get_attribute('outerHTML')

# Close the WebDriver
driver.quit()

# Parse with BeautifulSoup
soup = BeautifulSoup(main_html, "html.parser")

# REMOVE THE BROCHURE AND COUNTRIES SECTIONS
unwanted_section = soup.find("div", class_="hero-article__info")
if unwanted_section:
    unwanted_section.decompose()  


# REMOVING THE ACADEMIC CALENDER (6) SECTIONS
articles = soup.find_all("article", class_="content__article")
indices_to_remove = [5]  # 0-based index for 5th, 6th articles
for index in indices_to_remove:
    if index < len(articles):
        articles[index].decompose()
    

# Extract only the text, removing extra whitespace
main_text = soup.get_text(separator="\n", strip=True)
# Print extracted text
print(main_text)

# Sanitize posgrad_master_degree_name to ensure it's a valid file name
PhD_degree_name = PhD_url.split('/')[-2]
sanitized_name = sanitize_filename(PhD_degree_name) 
# Construct file path with sanitized name
file_path = os.path.join(PhD_dir, f"{sanitized_name}_main_course_extracted_text.txt")

# Write extracted text to the file
with open(file_path, "w", encoding="utf-8") as file:
    file.write(f"Text from {PhD_url}:\n{main_text}\n\n")


Doctoral Program in Information Management
Doctoral Program in Information Management
en
Education
Programs
Doctoral Program in Information Management
Doctoral Program in Information Management
The Doctoral Program in Information Management aims to
promote knowledge and innovation in Information Management
, particularly in the research of its impact on wealth creation, improving the performance of organizations and the development of new methodologies and decision-making tools.
It provides advanced training through research, contributing to the progress of knowledge and the training of researchers, university teachers and highly qualified professionals.
The study cycle in Information Management is designed to include five areas:
Business Analytics;
Data-Driven Marketing;
Data Science;
Geoinformatics;
Information Systems.
This Doctoral Program is part of
NOVA Doctoral School
, which main goals are to develop Good Academic Practices and offer complementary training to its students, in f

## FIN
